# Adding dial plans
This one is based on Johannes' tool   
The intention here is to pull learned routes from an SME cluster, then based on UCM route prefix match the patterns with a trunk/route group in Webex

In [1]:
import os
import requests
import json        # just for pretty printing
from dotenv import load_dotenv

load_dotenv()
WEBEX_TOKEN = os.environ.get("WEBEX_TOKEN", 
                             # replace below if you don't have a .env
                             "NWVkODBmZWItMjM0Yy00ZDRlLWFjZmQtZWE4ZGNmN2Q1ZDVhOTFiY2UyYjQtNTNj_P0A1_0ec1bbe0-b5aa-454a-bb5b-aecc4e6bdb4c")
BASE_URI = "https://webexapis.com/v1"

headers = {
    "Authorization": "Bearer " + WEBEX_TOKEN, 
    "Content-Type": "application/json", 
    "Accept": "application/json"
}

In [2]:
try:
    response = requests.get(
        BASE_URI + "/telephony/config/premisePstn/trunks", 
        headers=headers
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    trunks = response.json()
    print(json.dumps(trunks, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

trunk = next(t for t in trunks["trunks"] 
             if t["name"] == "Europe Trunk")
print(trunk["id"])

200 OK
{
    "trunks": [
        {
            "id": "Y2lzY29zcGFyazovL3VzL1RSVU5LL2NjNzEyOWJkLWFmM2UtNDRiNC04ZjExLTk3ODdkYzIwNTg0ZA",
            "name": "Europe Trunk",
            "location": {
                "name": "Amsterdam",
                "id": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OLzg2MjU3ZTY3LTcxZDItNDZkMy05ZWZhLTIxY2U2Mzg2NWNhZQ"
            },
            "inUse": true,
            "trunkType": "REGISTERING",
            "isRestrictedToDedicatedInstance": false
        }
    ]
}
Y2lzY29zcGFyazovL3VzL1RSVU5LL2NjNzEyOWJkLWFmM2UtNDRiNC04ZjExLTk3ODdkYzIwNTg0ZA


In [3]:
data = {
  "name": "EU-Trunk-ams",
  "routeId": trunk["id"],
  "routeType": "TRUNK"
}

try:
    response = requests.post(
        BASE_URI + "/telephony/config/premisePstn/dialPlans",
        headers=headers,
        json=data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    dial_plan = response.json()
    print(json.dumps(dial_plan, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

201 Created
{
    "id": "Y2lzY29zcGFyazovL3VzL0RJQUxfUExBTi9jMDVmMGFjMS03Y2Y4LTRiYmEtYmM2Mi00MGY2ZjA3OTQzZWM",
    "dialPatternErrors": []
}


Now we have a dialplan with the destination set to our (Local Gateway) trunk, but it doesn't have any patterns still 

With these SQL queries and AXL executeSQLQuery, we can find learned routes on an SME cluster 
```sql
select peerid,routestring from remoteclusteruricatalog
select remotecatalogkey_id,remoteclusteruricatalog_peerid from remotecatalogkey
```

From here we can create a list of route strings with corresponding catalog like this
> eur.cisco.com,+322702998X   
> amer.cisco.com,+1999888123X

then map the catalog to a dial-plan   
e.g. our dialplan "EU-Trunk-ams" we create above will route all "eur.cisco.com" routes


In [4]:
dialPlanId = dial_plan["id"]
data = {
    "dialPatterns": [{
        "dialPattern": "+322702998X",
        "action": "ADD"
    }]
}

try:
    response = requests.put(
        BASE_URI + f"/telephony/config/premisePstn/dialPlans/{dialPlanId}/dialPatterns",
        headers=headers,
        json=data
    )
    response.raise_for_status()
    print(f"{response.status_code} {response.reason}")
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

206 Partial Content
